In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import time
import argparse

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=26,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), 
                attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
    
    
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
dataset_dir = "data"
bertmodel, vocab = get_pytorch_kobert_model()

movie_data = pd.read_csv(dataset_dir + '/movie_data.csv')

data_list = []
for q, label in zip(movie_data['plot'], movie_data['genre_index'])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=123)

In [ ]:
movie_data

In [ ]:
data_list[0]

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 10
learning_rate =  0.00005

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).cuda()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters()if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
start = time.time()
num_epochs = 10

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            metric_accuracy = loss.data.cpu().numpy()
            metric_loss = train_acc / (batch_id+1)
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, metric_accuracy, metric_loss))

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

    print("epoch {} test acc {}\n".format(e+1, test_acc / (batch_id+1)))


training_time = time.time() - start
print("training time : ", round(training_time, 2))

In [ ]:
def predict(predict_sentence):
      
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
      
    device = torch.device("cuda:0")
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        
        genre = None
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            genre_index = np.argmax(logits)
            
            if genre_index == 0:
                genre = "드라마"
            elif genre_index == 1:
                genre = "액션"
            elif genre_index == 2:
                genre = "코미디"
            elif genre_index == 3:
                genre = "애니메이션"
            elif genre_index == 4:
                genre = "멜로/로맨스"
            elif genre_index == 5:
                genre = "공포"
            elif genre_index == 6:
                genre = "다큐멘터리"
            elif genre_index == 7:
                genre = "범죄"
            elif genre_index == 8:
                genre = "스릴러"
            elif genre_index == 9:
                genre = "SF"
            elif genre_index == 10:
                genre = "판타지"
            elif genre_index == 11:
                genre = "모험"
            elif genre_index == 12:
                genre = "미스터리"
            elif genre_index == 13:
                genre = "가족"
            elif genre_index == 14:
                genre = "공연실황"
            elif genre_index == 15:
                genre = "에로"
            elif genre_index == 16:
                genre = "전쟁"
            elif genre_index == 17:
                genre = "뮤지컬"
            elif genre_index == 18:
                genre = "서스펜스"
            elif genre_index == 19:
                genre = "느와르"
            elif genre_index == 20:
                genre = "무협"
            elif genre_index == 21:
                genre = "서부"
            elif genre_index == 22:
                genre = "블랙코미디"
            elif genre_index == 23:
                genre = "서사"
            elif genre_index == 24:
                genre = "컬트"
            elif genre_index == 25:
                genre = "실험"


        print("prediction of genre : " + genre)

In [ ]:
sentence = "16만 팔로워를 가진 힙합 댄서 E.D.윈에게 첫눈에 반한 12살 소녀 미나는 운 좋게 오디션을 통과하고 E.D.윈의 댄스 크루에 들어간다. 공부와 달리 마음대로 되지 않는 몸에 인생 첫 좌절을 마주한 미나. 하지만 포기란 없다! 한때 춤으로 이름 좀 날렸던 할머니의 지도하에 남사친 마르쿠스와 비밀스러운 연습을 시작하는데… 함께라면 할 수 있어! ★오늘부터 댄싱퀸★"

predict(sentence)